In [ ]:
# import pandas as pd

# # 1️⃣ ── LOAD YOUR DATA ────────────────────────────────────────────────────────
# # Replace the file name/path as needed
# infile  = "/home/rohank__iitp/Work/niladri/dataset3/11conversation.csv"
# outfile = "/home/rohank__iitp/Work/niladri/dataset3/conversation.csv"

# df = pd.read_csv(infile)

# # 2️⃣ ── MAKE SURE THE DATA ARE IN TURN ORDER ────────────────────────────────
# df = df.sort_values(["conversation_id", "turn_no"]).reset_index(drop=True)

# # 3️⃣ ── IDENTIFY WHICH USER ROWS TO KEEP ─────────────────────────────────────
# # Shift the Speaker and Conversation_id columns “up” one row
# df["next_speaker"] = df["speaker"].shift(-1)
# df["next_conv"]    = df["conversation_id"].shift(-1)

# # User rows we **want to keep** are those whose next row
# #  • is from the SAME conversation and
# #  • has Speaker == "Agent"
# mask_user_keep = (
#     (df["speaker"] == "User") &
#     (df["next_conv"] == df["conversation_id"]) &
#     (df["next_speaker"] == "Agent")
# )

# # Agent rows are always kept
# mask_agent_keep = df["speaker"] == "Agent"

# # Combine the two conditions
# mask_keep = mask_agent_keep | mask_user_keep

# # 4️⃣ ── FILTER & SAVE ────────────────────────────────────────────────────────
# filtered_df = df[mask_keep].drop(columns=["next_speaker", "next_conv"])
# filtered_df.to_csv(outfile, index=False)

# print(f"Original rows: {len(df)}")
# print(f"Rows after filtering: {len(filtered_df)}")
# print(f"Cleaned file written to: {outfile}")


Original rows: 46
Rows after filtering: 43
Cleaned file written to: /home/rohank__iitp/Work/niladri/dataset3/conversation.csv


In [37]:
import pandas as pd

# Load the original CSV file
df = pd.read_csv("/home/rohank__iitp/Work/niladri/dataset3/conversation.csv")  # replace with actual path if not in current directory

# Prepare variables
output_rows = []
agent_responses = []

# Temporary variables to track user query and agent reply
current_user_query = None
current_conversation_id = None
record_id = 1

for index, row in df.iterrows():
    if row['speaker'] == 'User':
        agent_responses.append(row['id'])
        if current_user_query is not None:
            # Save previous user query without agent reply
            output_rows.append({
                'id': record_id,
                'conversation_id': current_conversation_id,
                'user_query': current_user_query,
                'agent_reply': ''
            })
            record_id += 1
        # Start new user query
        current_user_query = row['utterance']
        current_conversation_id = row['conversation_id']
    elif row['speaker'] == 'Agent':
        
        if current_user_query is not None:
            # agent_responses.append(row['id'])
            # Save user query and agent reply
            output_rows.append({
                'id': record_id,
                'conversation_id': current_conversation_id,
                'user_query': current_user_query,
                'agent_reply': row['utterance']
            })
            record_id += 1
            current_user_query = None  # Reset after pairing

# If last entry is a user query without an agent reply
if current_user_query is not None:
    output_rows.append({
        'id': record_id,
        'conversation_id': current_conversation_id,
        'user_query': current_user_query,
        'agent_reply': ''
    })

# Convert to DataFrame
output_df = pd.DataFrame(output_rows)

# Save to CSV
output_df.to_csv("/home/rohank__iitp/Work/niladri/dataset3/allexp/table_allexp.csv", index=False)

# Print or return list of agent responses
print(agent_responses)


[1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41]


In [38]:
len(agent_responses)

21

In [39]:
# import pandas as pd

# # Load the CSV file
# df = pd.read_csv('/home/rohank__iitp/Work/niladri/dataset3/allexp/table_allexp.csv')

# # Drop rows where 'agent_reply' is NaN or an empty string
# df_cleaned = df[df['agent_reply'].notna() & (df['agent_reply'].str.strip() != '')]

# # Save the cleaned DataFrame to a new CSV (or overwrite the original)
# df_cleaned.to_csv('/home/rohank__iitp/Work/niladri/dataset3/allexp/table_allexp.csv', index=False)




In [40]:
import pandas as pd
import json

df=pd.read_csv("/home/rohank__iitp/Work/niladri/dataset3/allexp/table_allexp.csv")  # replace with actual path if not in current directory
# Output list for matching answers
matching_answers = []

# Read and process the JSONL file
with open('/home/rohank__iitp/Work/niladri/dataset3/allexp/cleaned_output.jsonl', 'r') as f:
    for line in f:
        data = json.loads(line)
        # if data.get('id_json') in agent_responses:
        matching_answers.append(data.get('answer'))

df["all expterts response"]= matching_answers
df.to_csv("/home/rohank__iitp/Work/niladri/dataset3/allexp/table_allexp.csv", index=False)

In [95]:
import sys
sys.path.append('/DATA/rohan_kirti/.local/lib/python3.8/site-packages')

In [41]:
from rouge_score import rouge_scorer


In [42]:
import pandas as pd
from rouge_score import rouge_scorer

# Load CSV
df = pd.read_csv("/home/rohank__iitp/Work/niladri/dataset3/allexp/table_allexp.csv")  # Replace with your actual file path

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)

# Compute ROUGE scores
def compute_rouge(row):
    scores = scorer.score(row['all expterts response'], row['agent_reply'])
    return scores['rouge1'].fmeasure  # You can also use .precision or .recall

# Apply ROUGE calculation
df['rouge'] = df.apply(compute_rouge, axis=1)

# Save the updated DataFrame
df.to_csv("/home/rohank__iitp/Work/niladri/dataset3/allexp/table_allexp.csv", index=False)